In [ ]:
import pandas as pd
import json
from ipyfilechooser import FileChooser
import ipywidgets as widgets
from IPython.display import display
import os

# **Unsupported:** Labelbox To Yolo
Dit script is geschreven om annotaties van modellen handmatig te kunnen controleren. Wanneer een project data van een model kreeg met de labels in de afbeelding "geprint" of allen de labels en niet of deze klopte met de werkelijkheid moest iedere annotatie van het model handmatig gecontroleerd worden. Hiervoor werd [Labelbox](https://labelbox.com/) gebruikt. Labelbox bied de mogelijkheid label voorschriften te maken waarbij naast het label ook een sub-label geselecteerd moet worden voor een valide annotatie. Dit betekend dat je de annotaties en/of labels in Labelbox kan laden en voor iedere bounding box en/of object een nieuwe annotatie kan maken waarbij je eerst annoteert welk label iets moet krijgen en het sub-label gebruikte om aan te geven wat het model er van maakt. Net als bij [**Unsupported:** Labelbox To Yolo](LabelboxToYolo.ipynb) kun je dan de annotaties downloaden in de vorm van een Json file. Dit script is bedoelt om de annotaties uit de json lezen en op te slaan in een ExcelFile voor verdere resultaat verwerking. 

---
<blockquote style="border-left: 5px solid red; padding-left: 10px;">

## **Attentie!** 
<code> Update onderstaande cell voordat je de het script runt </code></blockquote>
 

In [ ]:
# Input map met de Json file met labels die je gedownload hebt van Labelbox.
JsonFile = r"pad/naar/jouw/Json/labels.json"
# Bijvoorbeeld: r"pad/naar/jouw/Json/labels.json"

# Output map of bestand waar de nieuwe Excel worden opgeslagen.
OutputLocatie = r"pad/naar/jouw/output/bestand.xlsx"
# Bijvoorbeeld: r"pad/naar/jouw/output/bestand.xlsx" OF r"pad/naar/jouw/output/folder"

---
## **Code**

In [ ]:
# Checkt of de outputlocatie een excel bestand is en maakt deze anders aan op die locatie
if os.path.exists(OutputLocatie) and not OutputLocatie.endswith(".xlsx"):
    OutputLocatie = os.path.join(OutputLocatie, "resultaten.xlsx")
if os.path.exists(OutputLocatie):
    with open(OutputLocatie, "w+") as Output:
        Output.write("")

if os.path.exists(JsonFile) and os.path.exists(OutputLocatie):
    # Leest de json in
    with open(JsonFile, "r") as File:
        Data = json.load(File)
    
    # Loopt over alle annotaties in het json heen en slaat voor iedere annotatie de volgende data op:
    # "External ID"(Naam afbeelding voor upload)
    # ["title"] (Hooft label)
    # ["classifications"][0]["answer"]["title"] (sub-label)
    output_data = []
    for foto in Data:
        FileName = foto["External ID"]
        for soort in foto["Label"]["objects"]:
            if "classifications" in soort:
                Insect = soort["title"]
                Voorspelling = soort["classifications"][0]["answer"]["title"]
                output_data.append([FileName, Insect, Voorspelling])

    df = pd.DataFrame(output_data, columns=['Foto', 'Annotatie', 'Voorspelling'])
    df.to_excel(OutputLocatie, index=False, engine='xlsxwriter')
else:
    print("Zorg ervoor dat de opgegeven paden voor JsonFile en OutputLocatie kloppen en bestaan.")
